<a href="https://colab.research.google.com/github/darshan12x/projects/blob/mainbranch/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tweepy
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import numpy as np 
import matplotlib.pyplot as plt 
import string
import nltk
import warnings
import seaborn as sns
warnings.filterwarnings("ignore", category=DeprecationWarning)
nltk.download('vader_lexicon')

%matplotlib inline

In [0]:
#My Twitter API Authentication Variables
consumer_key = ' '
consumer_secret = ' '
access_token = ' '
access_token_secret = ' '

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [98]:
tweets = api.search('Irfan Khan', count=200) # search term 

# 'iso_language_code': 'en'
# 'location': ''
# 'followers_count': 41
# 'verified': False
# 'retweeted': False

data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

display(data.head(10))

,Tweets
0,RT @TheSquind: Na Fateha na Darood means you w...
1,RT @YUVSTRONG12: I know the journey I know the...
2,RIP IRFAN KHAN. 🙏\nSADDENED BY THE NEWS 😥 http...
3,RT @ZairaWasimmm: Platelets required for a 21 ...
4,RT @KhanSahiba01: R.I.P😔💔... Today's irfan kha...
5,"May Allah bless your soul, Sir. Rest in Love, ..."
6,वैसे तो दुनिया में आते हैं सब मरने के लिए\nअसल...
7,You'll be missed Sir 😓\nYou'll forever remain ...
8,RT @AmitShah: Anguished over the sad news of I...
9,RT @YUVSTRONG12: I know the journey I know the...


In [99]:
# tidy up tweets
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

# remove twitter handles (@user)
data['tidy_tweet'] = np.vectorize(remove_pattern)(data['Tweets'], "@[\w]*")

# remove links
data['tidy_tweet'] = np.vectorize(remove_pattern)(data['tidy_tweet'],"(\w+:\/\/\S+)")

# remove special characters, numbers, punctuations
data['tidy_tweet'] = data['tidy_tweet'].str.replace("([^0-9A-Za-z \t])", " ")

# removing short words
data['tidy_tweet'] = data['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

display(data.head(10))

,Tweets,tidy_tweet
0,RT @TheSquind: Na Fateha na Darood means you w...,Fateha Darood means you won get peace Hell wai...
1,RT @YUVSTRONG12: I know the journey I know the...,know the journey know the pain and know fought...
2,RIP IRFAN KHAN. 🙏\nSADDENED BY THE NEWS 😥 http...,RIP IRFAN KHAN SADDENED THE NEWS
3,RT @ZairaWasimmm: Platelets required for a 21 ...,Platelets required for year old lady suffering...
4,RT @KhanSahiba01: R.I.P😔💔... Today's irfan kha...,Today irfan khan more Allah pak irfan khan mag...
5,"May Allah bless your soul, Sir. Rest in Love, ...",May Allah bless your soul Sir Rest Love Irfan ...
6,वैसे तो दुनिया में आते हैं सब मरने के लिए\nअसल...,lines IrfanKhan RIP Irfan Khan
7,You'll be missed Sir 😓\nYou'll forever remain ...,You missed Sir You forever remain our hearts R...
8,RT @AmitShah: Anguished over the sad news of I...,Anguished over the sad news Irfan Khan demise ...
9,RT @YUVSTRONG12: I know the journey I know the...,know the journey know the pain and know fought...


In [0]:
# Alternate func
# regex to strips URL, punctuations, usernames and any non-alphanumeric chars. Also separates words with single space. 
""" 
def clean_tweet(tweet):
  return ' '.join(re.sub("@[\w]*|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())

data['tidy_tweet'] = pd.DataFrame(data=[clean_tweet(tweet.text) for tweet in tweets], columns=['Tweets'])
data['tidy_tweet'] = data['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
display(data.head(10))

"""

In [101]:
sid = SentimentIntensityAnalyzer()
listy = []

for index, row in data.iterrows():
  ss = sid.polarity_scores(row["tidy_tweet"])
  listy.append(ss)
  
se = pd.Series(listy)
data['polarity'] = se.values

display(data.head(100))

,Tweets,tidy_tweet,polarity
0,RT @TheSquind: Na Fateha na Darood means you w...,Fateha Darood means you won get peace Hell wai...,"{'neg': 0.165, 'neu': 0.576, 'pos': 0.259, 'co..."
1,RT @YUVSTRONG12: I know the journey I know the...,know the journey know the pain and know fought...,"{'neg': 0.202, 'neu': 0.614, 'pos': 0.184, 'co..."
2,RIP IRFAN KHAN. 🙏\nSADDENED BY THE NEWS 😥 http...,RIP IRFAN KHAN SADDENED THE NEWS,"{'neg': 0.405, 'neu': 0.595, 'pos': 0.0, 'comp..."
3,RT @ZairaWasimmm: Platelets required for a 21 ...,Platelets required for year old lady suffering...,"{'neg': 0.171, 'neu': 0.829, 'pos': 0.0, 'comp..."
4,RT @KhanSahiba01: R.I.P😔💔... Today's irfan kha...,Today irfan khan more Allah pak irfan khan mag...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
...,...,...,...
95,RT @AmitShah: Anguished over the sad news of I...,Anguished over the sad news Irfan Khan demise ...,"{'neg': 0.238, 'neu': 0.645, 'pos': 0.117, 'co..."
96,RT @Boii_is_Lazy: Bollywood lost a Gem 💔\nI am...,Bollywood lost Gem not his Die Hard fan but ca...,"{'neg': 0.271, 'neu': 0.53, 'pos': 0.199, 'com..."
97,@ShraddhaKapoor Irfan khan sir love u give pea...,Irfan khan sir love give peace the soul god,"{'neg': 0.0, 'neu': 0.38, 'pos': 0.62, 'compou..."
98,https://t.co/1hJssVNKCV Bollywood Hollywood St...,Bollywood Hollywood Star Irfan Khan More,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [102]:
#  tokenize all the cleaned tweets in our dataset
tokenized_tweet = data['tidy_tweet'].apply(lambda x: x.split())

tokenized_tweet.head()

0    [Fateha, Darood, means, you, won, get, peace, ...
1    [know, the, journey, know, the, pain, and, kno...
2              [RIP, IRFAN, KHAN, SADDENED, THE, NEWS]
3    [Platelets, required, for, year, old, lady, su...
4    [Today, irfan, khan, more, Allah, pak, irfan, ...
Name: tidy_tweet, dtype: object